# Frequency distributions

One of the most fundamental forms of analysis is to look at
the distribution of values of some quantitative property in relation
to a set of data samples. One may then try to form an explanation
of why the values are distributed in this way.

From a mathematical point of view, one can describe a frequency
distribution in terms of some mathematical function. Many techniques
have been developed for finding functions that fit a given set of
values or for determining how closely a function fits a set of
values. However, there is an infinite variety of possible functions,
and it is always possible to construct a function that will exactly
fit any frequency distribution.
Hence, one is usually looking for kinds of function that give some explanation of why the frequency distribution occurs. Typically one wants to find a function for
which there is a statistical and/or physical basis. For example:

* an uniformly distributed random variable, such as the roll of a dice
  or the selection of winning numbers in a lottery;
* a Gausian distribution, which can come from the cumulative result of many underlying
  random factors;
* the result of an iterative process, consisting of steps that
  have a random outcome (which might be modelled as a
  <a target="_blank" href="https://en.wikipedia.org/wiki/Markov_chain">Markov Chain</a>
  or a <a target="_blank" href="https://en.wikipedia.org/wiki/Survival_function">survival function</a>);
* a function depending on some spatial or physical property
  (e.g. proximity to a radiation source).

More generally, a frequency distribution may result from a
combination of several different factors, and thus explanation
may require a complex model incorporating several types
of function.

The kinds of analysis just mentioned go well beyond the scope of
this course. In this notebook we will not try to analyse
data and construct a model for its frequency distribution, instead we shall
go the other way round: we will use Python both
to generate and to visualise various types of random function.
This will give you some insight into what kinds of frequency
distribution might give rise to the data that you encounter, and hence
guide you towards its analysis. It will also illustrate a number
of useful programming techniques.


In [ ]:
# Packages used in this notebook.
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import random
import math

## Normal (Gaussian) distributions

Probably the best known frequency distribution is the
<a target="_blank" href="https://en.wikipedia.org/wiki/Normal_distribution">*normal* distribution</a>,
which is also often called the *Gaussian* distribution (after the mathematician
<a target="_blank" href="https://en.wikipedia.org/wiki/Carl_Friedrich_Gauss">Carl Gauss</a>).
Roughly speaking, a normal distribution is a frequency distribution where
the values are concentrated about the average value, with values being less
common the further they are from the average. Truly normal distributions
are always symmetrical about the mean value but can vary in how quickly
the frequencies decrease for values away from the mean, and this spread is characterised
by the variance (the square of the standard deviation). A normal distribution is determined by its mean and variance.

We could use the Python package `numpy` to directly generate normal distributions.
However, from a programming point of view, it will be more interesting and
instructive to consider the discrete approximation of a normal distribution generated by rolling several dice and adding their scores.
Dice rolling scores are 'discrete' in the sense that the score is always
a whole number, whereas the term 'normal distribution' properly applies
only to continuous valued functions.

The following code defines a function `dice_total`, which returns a value which one might obtain by rolling `num` dice, where the number of sides on each dice is `sides`, and the returned value is the sum of the values on the `num` dice. The distribution
of returned values we obtain should correspond exactly to what we would get from
actually rolling physical dice (assuming they are fair unbiased dice).

In [ ]:
def dice_total(num, sides):
    total = 0
    for roll in range(num):
        total += random.randint(1,sides)
    return total

One simple way of investigating the distribution of values of this function is by calling it many times and making a _scatter plot_ of the values obtained.
It is easy to do that using the `matplotlib` package. In the plot below the
$x$ axis is the number of the function call sample (in a sequence of 1000 samples)
and the $y$ axis gives the value of the sample.
Alongside the scatter plot we display a box-plot of the data which gives an alternative way of visualising how the values are distributed.
Remember that another easy way to get information about your data is to import it into a `pandas` `DataFrame` and then use the methods provided by `DataFrame` objects to determine or display properties of the distribution.
In particular, the `describe` method of DataFrames summarises some basic statistical properties.


In [ ]:
# Create the data by calling dice_total 10,000 times
# The sum of 10 dice, each with 20 faces is computed by dice_total
dice_roll_samples = [dice_total(10,20) for _ in range(10000)]

fig = plt.figure(figsize=(8,4))
gs = GridSpec(1, 4, figure=fig)

ax1 = fig.add_subplot(gs[0, :3])
ax2 = fig.add_subplot(gs[0, 3:])

ax1.scatter(list(range(10000)), dice_roll_samples, s=5)

ax2.boxplot(dice_roll_samples)

ax2.spines['right'].set_color('none')
ax2.spines['bottom'].set_color('none')
ax2.spines['top'].set_color('none')
ax2.spines['left'].set_color('none')
ax2.get_xaxis().set_visible(False)
ax2.get_yaxis().set_visible(False)

fig.tight_layout()


### Making a frequency dictionary
A way of representing a frequency distribution that is convenient for many
programming tasks is to store it in a dictionary. The following code builds
such a dictionary by iterating through the list of values. When it finds
a new value it creates a new dictionary key for that value and associates
that key with the number 1 (meaning that one occurrence of that value has
been found). If it comes to a value for which a key already exists it
adds one to the number associated with that key. Hence, it builds up
a dictionary in which each observed value is a key, which is associated
with the number of times that value was observed.
This dictionary can then easily be used to plot a bar chart showing
the distribution.


In [ ]:
frequencies = {}
for val in dice_roll_samples:
    if val in frequencies:
        frequencies[val] += 1
    else:
        frequencies[val] = 1

# plot the distribution
fig, ax = plt.subplots()
ax.bar(frequencies.keys(), frequencies.values())

## A General function for plotting random function frequencies

Given that plotting the frequency distribution of the values of a random function
may be interesting for many different random functions that we may be interested in, it will be useful to define a general plotting function that can do this.
The `get_freq_of_random_function()` function defined below can be given any function
and a number of samples whose values we want to plot. For this to work as intended,
the function argument `random_function` must have been defined and should be a random function that returns a single numerical value.


In [ ]:
def get_freq_of_random_function(n_samples, random_function, *func_args):
    """
        calls a provided function for generating random numbers

        Parameters:
            n_samples (int): the number of times to call the function
            random_function: a function to generate a single item of data
            *func_args: any arguments required for random_function

        Returns:
            dictionary of values: frequencies
    """
    freq = {} # initialise

    for tests in range(n_samples):
        # call the random function and pass in any arguments provided for it
        result = random_function(*func_args)
        # update frequency record with this result
        if result in freq:
            freq[result] += 1
        else:
            freq[result] = 1

    return freq

def plot_freq_dist(frequencies, plot_title):
    """
        bar chart of given frequency distribution

        Parameters:
            frequencies dict(float: int): dictionary of value: frequency of value
            plot_title (str): title of the plot

        Returns:
            matplotlib figure
    """
    # prepare figure area
    fig, ax = plt.subplots()
    ax.set_title(plot_title)

    # plot bars
    ax.bar(frequencies.keys(), frequencies.values())

    # axes labels
    ax.set_xlabel('value')
    ax.set_ylabel('frequency')

    return fig

Now let's test out the functions we've defined above with the `dice_total` function that we created earlier.

In [ ]:
frequencies = get_freq_of_random_function(1000, dice_total, 10, 6)
fig = plot_freq_dist(frequencies, 'dice_total')
plt.show()

## Bimodal distributions

Suppose we are investigating a species of <a target="_blank" href="https://en.wikipedia.org/wiki/Flea">flea</a>.
All the fleas we observe are very small, dark purplish in colour (<a target="_blank" href="https://en.wikipedia.org/wiki/Puce">puce</a>, in fact) and jump about a lot.
We wonder what is the average height a flea can jump and, after measuring
many flea hop heights, we find that the average is about 31cm.
But what is strange is that nearly all the hops we measure are either
well below or well above 31 cm. What could be the explanation of this?

A possible explanation of this is that the fleas we are observing are
not all of the same type. There may be sub-species or genetic
variants with different jumping capabilities.

Let us try to simulate this situation using a random function in
which to get a value we first randomly choose a type, and then,
according to the chosen type, we generate a random value, such
that the distribution of values depends on the type.
For simplicity, we consider a case where there are two types
of fleas ("normal" and "super"). The jump height distribution of "normal"
fleas is approximated by the random function generated by rolling
5 six-sided dice, and the jump height distribution for "super" fleas
is approximated by 8 ten-sided dice. Hence, we can define a random
`flea_hop_height` function and plot its frequency distribution
with the following code:


In [ ]:
def flea_hop_height():
    flea_type = random.choice(["normal", "super"])
    if flea_type == "normal":
        return dice_total(5,6)
    if flea_type == "super":
        return dice_total(8,10)

frequencies = get_freq_of_random_function(1000,flea_hop_height)
fig = plot_freq_dist(frequencies, 'flea_hop_height')

Here we see that the two-type model gives us exactly the distribution we had observed, which supports the argument that there are indeed two types of flea. We should also investigate other explanations, for example perhaps the size of the flea's just depends on how long since its last jump, or perhaps it performs a large jump every so often.

## Exponential survival function
Consider a betting game based on the following rules:

1. You pay me £10 to play.
2. You play by throwing a 6-sided dice until you roll a 1 –
    * if you roll 2 or higher, I give you £1 and you can roll again
    * if you roll a 1, the game is over.

Should you play this game?

The calculation of the expected payoff is somewhat complex, but it is quite easy to simulate the game in Python code. The function below models this process when the value of risk is set to 1/6, i.e. the probability of rolling a 1. The function `random.random()` produces uniformaly distributed random numbers between 0 and 1, so if the number it produces is greater than 1/6, it's equivalent to rolling a 2 or higher. If it's less than or equal to 1/6 it's equivalent to rolling a 1.

We then use our general function to call sample from `constant_risk_survival` 1000 times and produce a frequency plot.

In [ ]:
def constant_risk_survival(risk):
    n = 0
    while random.random() > risk:
        n+=1
    return n

number_of_samples=1000

frequencies = get_freq_of_random_function(number_of_samples, constant_risk_survival, 1/6)
fig = plot_freq_dist(frequencies, 'constant_risk_survival')
plt.show()  

Now let's print some statistical information about this distribution. Here we interpret payoff as the amount of money you win in the game, minus the cost to play.

In [ ]:
# Compute mean and standard deviation of payoff
payoff_mean=0
payoff_std=0

for payoff, count in frequencies.items():
    payoff_mean+=(payoff-10)*count
    payoff_std+=(payoff-10)**2*count
    
payoff_mean/=number_of_samples
payoff_std=math.sqrt(payoff_std/(number_of_samples-1))

# Print statistics:
print(f'Mean payoff: {payoff_mean}')
print(f'Payoff standard deviation: {payoff_std}')
print(f'Maximum payoff: £{max(frequencies.keys())-10}')
print(f'Percentage of time you lose: {100*sum(frequencies[i] for i in range(10))/number_of_samples:.2f}%')

In the code above, try increasing the `number_of_samples` - depending on your computer, you may be able to increase this to a million or more. 

You should find that the larger the `number_of_samples`, the closer the mean payoff is to -£5, i.e. on average you will lose money playing this game. Moreover, the statistics above show that you will lose money more than 80% of the time. However, the standard deviation is quite large, so the largest payoff could be more than £60.

Thus our simulation shows that this probably isn't a good game to play.

It is possible to confirm our simulated result mathematically. Assuming the dice is fair, then the probability of making $k$ rolls before getting a 1 is given by probability mass function of the [negative binomial distribution](https://en.wikipedia.org/wiki/Negative_binomial_distribution). The probability of rolling a 1 is $p=1/6$, and the mean number of rolls before getting a 1 is given by $(1-p)/p=5$. Since you get £1 back for each of these rolls, on average you get £5 back, and so on average you lose £5.

## A diminishing risk survival function

According to an exponential survival function, the chance of surviving from each time step to the next is constant. This is sometimes called a _memoryless_ survival function, because survival at each point does not depend on the previous history
(e.g. each die
roll is independent of the values of  previous rolls and of the number of rolls that
have been made).
However, it is arguable that in many cases the survival rate will change as time
progresses. For instance, as a person gets older they gradually get weaker and
less able to withstand illnesses and accidents. Hence, unfortunately, our risk
of dying in a given year increases as we get older.

The following code is a simple implementation of a survival function where the
risk factor changes by a constant factor at each time step.


In [ ]:
import random, math
def changing_risk_survival(risk, change):
    n = 0
    while random.random() > risk:
        n+=1
        risk = risk*change
        if n > 100:
            break
    return n

frequencies = get_freq_of_random_function(100, changing_risk_survival, 0.5, 0.8)
fig = plot_freq_dist(frequencies, 'changing_risk_survival')
plt.show()  


Here we see a very distinctive distribution in which the most frequent survival times are very short, and yet there are a significant number of cases that have a very high value. Notice that the bar for the value 101, represents all cases where the survival time was greater than 100. Many of these might have actually had much higher values, except that the `break` statement in `changing_risk_survival` stops it from counting past 101.

**What may happen if we remove the `break` command?**

Since the risk factor goes down every time we go round the loop, the chance
of a random number being lower than the risk gets smaller and smaller.
The program could run round the loop for an extremely long time.

**Can you think of a real situation that might be a close fit to the
diminishing risk survival function just presented?**

An good example of such a situation would be a video game, which is
very tough when you first start playing, but each time you play you
develop more skills and your character gets more resilient.
A similar example could be animals that are born into a harsh
environment, but each day they get stronger and better adapted
to survive.

**When doing data analysis on a phenomenon that has the characteristics
of a diminishing risk survival function, would it be a good idea to
treat very high values as outliers, and remove them from the sample
set as they are not typical compared with average values?**

It depends what kind of analysis you want to perform. But in most cases
it is probably not a good idea to treat the high values as outliers,
since they are a significant and interesting feature of the underlying
frequency distribution.